In [1]:
from prettytable import PrettyTable as PT
import re

# Parsing seqs files

In [2]:
CDH_seqs = {}
# Reading CDHIT transcripts to clusterID
clstr_file = open("cdhit/cdhit_pc2_transcripts_80.fa.clstr", "r")
clstr_data = clstr_file.read()
clstr_file.close()

rep = {"\t": ",", "at +/": "", "at -/": "",
       "...": ",", "nt": "", "%": "", " ": ""}
rep = dict((re.escape(k), v) for k, v in rep.iteritems())
pattern = re.compile("|".join(rep.keys()))
clstr_data = pattern.sub(lambda m: rep[re.escape(m.group(0))], clstr_data)
all_clusters = clstr_data.split(">Cluster")

cdhit_tr_cluster = {}
for i in range(1, len(all_clusters), 1):
    cluster = all_clusters[i]
    cluster = cluster.split("\n")
    cluster_id = cluster[0]
    for item in cluster[1:-1]:
        if cluster_id in CDH_seqs:
            CDH_seqs[cluster_id] += 1
        else:
            CDH_seqs[cluster_id] = 1


"""
Parsing kCluster Clusters file
"""
kCl_seqs = {}
with open("clusters_c66.0_66%_clusters.tsv", "r") as kCL:
    next(kCL)
    for line in kCL:
        cline = line.split()
        cluster_id = cline[0]
        no_of_seqs = len(cline[1].split(","))
        kCl_seqs[cluster_id] = no_of_seqs

## <b>Pairwise Seqs Count : kCl -> CDHIT & CDHIT -> kCl</b>

In [3]:
empty_d = {"CC":0, "IC":0, "CM":0, "IM":0}
kCl_to_CDH = {"CC":dict(empty_d), "IC":dict(empty_d), "CM":dict(empty_d), "IM":dict(empty_d)}
CDH_to_kCl = {"CC":dict(empty_d), "IC":dict(empty_d), "CM":dict(empty_d), "IM":dict(empty_d)} # For CDHIT -> kCl
with open("uniq_bio_assess.tsv", 'r') as tsv:
    next(tsv)
    for line in tsv:
        line = line.split()
        kCl_ID = line[0]
        kCl_type = line[1]
        CDH_ID = line[2]
        CDH_type = line[3]
        kCl_to_CDH[kCl_type][CDH_type] += kCl_seqs[kCl_ID]
        CDH_to_kCl[CDH_type][kCl_type] += CDH_seqs[CDH_ID] # For CDHIT -> kCl

## Generating Sumary tables

In [4]:
kClToCDHIT = PT()
kClToCDHIT.field_names = ["kCl/CDH","CC","IC","CM","IM"]
total_clusters1 = 0
TYPES = ["CC","IC","CM","IM"]
for TYPE in TYPES:
    row = []
    for T in TYPES:
        total_clusters1 += kCl_to_CDH[TYPE][T]
        row.append(kCl_to_CDH[TYPE][T])
    kClToCDHIT.add_row([TYPE]+row)

In [5]:
CDHTokCl = PT()
CDHTokCl.field_names = ["CDH/kCl","CC","IC","CM","IM"]
total_clusters2 = 0
TYPES = ["CC","IC","CM","IM"]
for TYPE in TYPES:
    row = []
    for T in TYPES:
        total_clusters2 += CDH_to_kCl[TYPE][T]
        row.append(CDH_to_kCl[TYPE][T])
    CDHTokCl.add_row([TYPE]+row)


## <hr> Summaries

In [6]:
print "kCl -> CDHIT\n", kClToCDHIT
print "Total Seqs: ", total_clusters1
print "~" * 65, "\n"
print "CDHIT -> kCl\n", CDHTokCl
print "Total Seqs: ", total_clusters2

kCl -> CDHIT
+---------+-------+-------+-----+------+
| kCl/CDH |   CC  |   IC  |  CM |  IM  |
+---------+-------+-------+-----+------+
|    CC   | 23032 | 75668 | 429 | 1951 |
|    IC   |  1152 | 71426 |  10 | 715  |
|    CM   |   28  |  729  | 495 | 1402 |
|    IM   |   7   |  719  |  26 | 1596 |
+---------+-------+-------+-----+------+
Total Seqs:  179385
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

CDHIT -> kCl
+---------+-------+-------+-----+-----+
| CDH/kCl |   CC  |   IC  |  CM |  IM |
+---------+-------+-------+-----+-----+
|    CC   | 22861 |  2027 |  10 |  3  |
|    IC   | 26396 | 31771 |  80 |  66 |
|    CM   |  1376 |   45  | 625 |  43 |
|    IM   |  3410 |  1419 | 944 | 707 |
+---------+-------+-------+-----+-----+
Total Seqs:  91783
